# Workshop Notebook 2: Observability Part Validation Rules

In the previous notebooks you uploaded the models and artifacts, then deployed the models to production through provisioning workspaces and pipelines. Now you're ready to put your feet up! But to keep your models operational, your work's not done once the model is in production. You must continue to monitor the behavior and performance of the model to insure that the model provides value to the business.

In this notebook, you will learn about adding validation rules to pipelines.

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.

In [17]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework


from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import datetime
import time

# used for unique connection names

import string
import random

In [18]:
## convenience functions from the previous notebooks
## these functions assume your connection to wallaroo is called wl

## convenience functions from the previous notebook

# return the workspace called <name> through the Wallaroo client.
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
            return workspace
    # if no workspaces were found
    if workspace==None:
        raise KeyError(f"Workspace {name} was not found.")
    return workspace


# returns the most recent model version in a workspace for the matching `model_name`
def get_model_version(model_name, workspace):
    modellist = workspace.models()
    model_version = [m.versions()[-1] for m in modellist if m.name() == model_name]
    # if no models match, return None
    if len(modellist) <= 0:
        raise KeyError(f"Model {mname} not found in this workspace")
        return None
    return model_version[0]

# get a pipeline by name in the workspace
def get_pipeline(pipeline_name, workspace):
    plist = workspace.pipelines()
    pipeline = [p for p in plist if p.name() == pipeline_name]
    if len(pipeline) <= 0:
        raise KeyError(f"Pipeline {pipeline_name} not found in this workspace")
        return None
    return pipeline[0]


## Login to Wallaroo

Retrieve the previous workspace, model versions, and pipelines used in the previous notebook.

In [19]:
## blank space to log in 

## blank space to log in 

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-07"

workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

embeddar_model_name = 'embedder'

embeddar_model_version = get_model_version(embeddar_model_name, workspace)

sentiment_model_name = 'sentiment'

sentiment_model_version = get_model_version(sentiment_model_name, workspace)

pipeline_name = 'imdb-reviewer'

pipeline = wl.get_pipeline(pipeline_name)

# display the current workspace, models, and pipelines

display(wl.get_current_workspace())

display(embeddar_model_version)

display(sentiment_model_version)

display(pipeline)


{'name': 'workshop-workspace-john-07', 'id': 23, 'archived': False, 'created_by': 'aa707604-ec80-495a-a9a1-87774c8086d5', 'created_at': '2023-09-19T15:46:40.207218+00:00', 'models': [{'name': 'embedder', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 19, 15, 48, 20, 528237, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 19, 15, 48, 20, 528237, tzinfo=tzutc())}, {'name': 'sentiment', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 19, 15, 48, 21, 222551, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 19, 15, 48, 21, 222551, tzinfo=tzutc())}], 'pipelines': [{'name': 'imdb-reviewer', 'create_time': datetime.datetime(2023, 9, 19, 15, 48, 25, 276825, tzinfo=tzutc()), 'definition': '[]'}]}

Name,embedder
Version,28d10148-e1e7-4bc4-9d40-2e0ea722f9b0
File Name,embedder.onnx
SHA,d083fd87fa84451904f71ab8b9adfa88580beb92ca77c046800f79780a20b7e4
Status,ready
Image Path,None
Updated At,2023-19-Sep 15:48:20


Name,sentiment
Version,970816c6-d167-427e-a684-15f4caf5aa5a
File Name,sentiment_model.onnx
SHA,3473ea8700fbf1a1a8bfb112554a0dde8aab36758030dcde94a9357a83fd5650
Status,ready
Image Path,None
Updated At,2023-19-Sep 15:48:21


name,imdb-reviewer
created,2023-09-19 15:48:25.276825+00:00
last_updated,2023-09-19 15:48:41.266497+00:00
deployed,False
tags,
versions,"a5f1d60f-d76b-4c9f-b047-9d8ec04699e3, 8c40e310-b6bc-4f77-8b73-258e0f23cced, bed8c8d4-a67d-4795-b6ee-fcd7d8535bb3"
steps,embedder


## Deploy the Pipeline

Add the model version as a pipeline step to our pipeline, and deploy the pipeline.  You may want to check the pipeline steps to verify that the right model version is set for the pipeline step.

In [20]:
## blank space to get your pipeline and run a small batch of data through it to see the range of predictions

pipeline.clear()
pipeline.add_model_step(embeddar_model_version)
pipeline.add_model_step(sentiment_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

multiple_result = pipeline.infer_from_file('../data/test_data.df.json')
display(multiple_result)
display(len(multiple_result))


,time,in.tensor,out.dense_1,check_failures
0,2023-09-19 15:52:23.819,"[1607.0, 2635.0, 5749.0, 199.0, 49.0, 351.0, 16.0, 2919.0, 159.0, 5092.0, 2457.0, 8.0, 11.0, 1252.0, 507.0, 42.0, 287.0, 316.0, 15.0, 65.0, 136.0, 2.0, 133.0, 16.0, 4311.0, 131.0, 286.0, 153.0, 5.0, 2826.0, 175.0, 54.0, 548.0, 48.0, 1.0, 17.0, 9.0, 183.0, 1.0, 111.0, 15.0, 1.0, 17.0, 284.0, 982.0, 18.0, 28.0, 211.0, 1.0, 1382.0, 8.0, 146.0, 1.0, 19.0, 12.0, 9.0, 13.0, 21.0, 1898.0, 122.0, 14.0, 70.0, 14.0, 9.0, 97.0, 25.0, 74.0, 1.0, 189.0, 12.0, 9.0, 6.0, 31.0, 3.0, 244.0, 2497.0, 3659.0, 2.0, 665.0, 2497.0, 63.0, 180.0, 1.0, 17.0, 6.0, 287.0, 3.0, 646.0, 44.0, 15.0, 161.0, 50.0, 71.0, 438.0, 351.0, 31.0, 5749.0, 2.0, 0.0, 0.0]",[0.37142318],0
1,2023-09-19 15:52:23.819,"[10.0, 25.0, 107.0, 11.0, 17.0, 2.0, 10.0, 119.0, 21.0, 456.0, 15.0, 11.0, 17.0, 6388.0, 10.0, 59.0, 21.0, 101.0, 41.0, 167.0, 5.0, 1447.0, 85.0, 10.0, 78.0, 21.0, 37.0, 11.0, 701.0, 2.0, 91.0, 2080.0, 4786.0, 10.0, 78.0, 21.0, 37.0, 5.0, 847.0, 782.0, 6388.0, 85.0, 10.0, 78.0, 21.0, 388.0, 65.0, 1098.0, 135.0, 59.0, 10.0, 137.0, 5.0, 2317.0, 51.0, 10.0, 244.0, 137.0, 5.0, 2316.0, 39.0, 1.0, 2346.0, 4519.0, 2316.0, 2.0, 1.0, 2346.0, 4519.0, 23.0, 1.0, 6222.0, 10.0, 8289.0, 681.0, 1.0, 7508.0, 5235.0, 78.0, 21.0, 388.0, 1.0, 782.0, 1098.0, 40.0, 37.0, 69.0, 1614.0, 10.0, 77.0, 21.0, 1411.0, 1.0, 2317.0, 1185.0, 54.0, 548.0, 48.0, 10.0, 235.0]",[0.9655761],0
2,2023-09-19 15:52:23.819,"[8.0, 3637.0, 4293.0, 1.0, 4523.0, 2.0, 2869.0, 4402.0, 2312.0, 8937.0, 4895.0, 3.0, 1872.0, 2204.0, 4.0, 695.0, 8461.0, 40.0, 5.0, 76.0, 192.0, 275.0, 5.0, 3164.0, 2.0, 8.0, 1813.0, 6501.0, 24.0, 4095.0, 2.0, 61.0, 5182.0, 6.0, 484.0, 2.0, 257.0, 1276.0, 16.0, 1639.0, 369.0, 7.0, 7.0, 6.0, 32.0, 2497.0, 1147.0, 2.0, 573.0, 355.0, 17.0, 41.0, 32.0, 9111.0, 3729.0, 12.0, 492.0, 3.0, 376.0, 4.0, 501.0, 39.0, 2477.0, 40.0, 5.0, 76.0, 192.0, 275.0, 5.0, 815.0, 4119.0, 2.0, 109.0, 1238.0, 3422.0, 685.0, 5.0, 24.0, 8670.0, 1997.0, 8.0, 16.0, 894.0, 11.0, 106.0, 59.0, 27.0, 1.0, 2606.0, 6725.0, 3528.0, 4.0, 1.0, 2132.0, 1391.0, 2.0, 445.0, 20.0, 11.0, 62.0]",[0.07601619],0
3,2023-09-19 15:52:23.819,"[11.0, 19.0, 6.0, 364.0, 16.0, 3452.0, 7794.0, 2.0, 196.0, 105.0, 560.0, 3.0, 173.0, 5.0, 27.0, 4624.0, 8.0, 1.0, 93.0, 4.0, 65.0, 285.0, 83.0, 196.0, 105.0, 23.0, 52.0, 462.0, 2033.0, 228.0, 9.0, 251.0, 5.0, 64.0, 616.0, 802.0, 1445.0, 330.0, 1080.0, 19.0, 45.0, 2571.0, 2.0, 22.0, 23.0, 914.0, 5.0, 103.0, 3.0, 2273.0, 19.0, 148.0, 4451.0, 124.0, 303.0, 5.0, 25.0, 3.0, 125.0, 4453.0, 1274.0, 10.0, 207.0, 2784.0, 2571.0, 18.0, 15.0, 1.0, 695.0, 43.0, 47.0, 11.0, 215.0, 3.0, 436.0, 131.0, 285.0, 709.0, 187.0, 23.0, 21.0, 1.0, 2154.0, 4.0, 1.0, 201.0, 19.0, 1184.0, 40.0, 1.0, 7076.0, 5002.0, 109.0, 686.0, 2745.0, 300.0, 7.0, 7.0, 14.0, 15.0]",[0.24645236],0
4,2023-09-19 15:52:23.819,"[10.0, 61.0, 927.0, 20.0, 63.0, 52.0, 49.0, 105.0, 2.0, 20.0, 2148.0, 1906.0, 58.0, 5517.0, 6.0, 5.0, 336.0, 81.0, 34.0, 178.0, 5.0, 64.0, 84.0, 105.0, 5.0, 1138.0, 65.0, 55.0, 2.0, 275.0, 6528.0, 7.0, 7.0, 10.0, 79.0, 178.0, 5.0, 567.0, 81.0, 3109.0, 65.0, 55.0, 20.0, 1240.0, 2.0, 178.0, 5.0, 1.0, 189.0, 12.0, 1.0, 164.0, 1323.0, 4.0, 131.0, 1240.0, 105.0, 188.0, 76.0, 242.0, 16.0, 9.0, 15.0, 52.0, 193.0, 72.0, 77.0, 166.0, 43.0, 34.0, 22.0, 23.0, 2.0, 77.0, 2299.0, 16.0, 43.0, 2187.0, 2.0, 7.0, 7.0, 11.0, 19.0, 692.0, 732.0, 80.0, 1.0, 1240.0, 2375.0, 7.0, 7.0, 1.0, 164.0, 2.0, 561.0, 6.0, 305.0, 42.0, 207.0, 3.0]",[0.08632833],0
5,2023-09-19 15:52:23.819,"[51.0, 22.0, 165.0, 30.0, 1.0, 1106.0, 2.0, 329.0, 535.0, 41.0, 9.0, 32.0, 1093.0, 272.0, 549.0, 4.0, 17.0, 263.0, 5.0, 327.0, 71.0, 48.0, 22.0, 76.0, 130.0, 92.0, 171.0, 276.0, 10.0, 329.0, 1.0, 2718.0, 15.0, 1.0, 82.0, 17.0, 443.0, 302.0, 14.0, 47.0, 68.0, 104.0, 99.0, 4.0, 11.0, 422.0, 622.0, 41.0, 1.0, 169.0, 55.0, 16.0, 196.0, 1889.0, 1840.0, 12.0, 66.0, 1316.0, 788.0, 8.0, 1139.0, 187.0, 883.0, 535.0, 41.0, 12.0, 17.0, 130.0, 10.0, 121.0, 10.0, 216.0, 11.0, 28.0, 2.0, 21.0, 12.0, 28.0, 2.

50

## Model Validation Rules

A simple way to try to keep your model's behavior up to snuff is to make sure that it receives inputs that it expects, and that its output is something that downstream systems can handle. This can entail specifying rules that document what you expect, and either enforcing these rules (by refusing to make a prediction), or at least logging an alert that the expectations described by your validation rules have been violated. As the developer of the model, the data scientist (along with relevant subject matter experts) will often be the person in the best position to specify appropriate validation rules.

In our house price prediction example, suppose you know that house prices in your market are typically in the range $750,000 to $1.5M dollars. Then you might want to set validation rules on your model pipeline to specify that you expect the model's predictions to also be in that range. Then, if the model predicts a value outside that range, the pipeline will log that one of the validation checks has failed; this allows you to investigate that instance further.

Note that in this specific example, a model prediction outside the specified range may not necessarily be "wrong"; but out-of-range predictions are likely unusual enough that you may want to "sanity-check" the model's behavior in these situations.

Wallaroo has functionality for specifying simple validation rules on model input and output values.

```python
pipeline.add_validation(<rulename>, <expression>)

```

Here, `<rulename>` is the name of the rule, and `<expression>` is a simple logical expression that the data scientist **expects to be true**.  This means if the expression proves **false**, then a `check_failure` flag is set in the inference results.

To add a validation step to a simple one-step pipeline, you need a handle to the pipeline (here called `pipeline`), and a handle to the model in the pipeline (here called `model`).  Then you can specify an expected prediction range as follows:

* Get the pipeline.
* Depending on the steps, the pipeline can be cleared and the sample model added as a step.
* Add the validation.
* Deploy the pipeline to set the validation as part of its steps

```python
# get the existing pipeline (in your workspace)
pipeline = get_pipeline("pipeline")

# you also need a handle to the model in this single-step pipeline.
# here are two ways to do it:
#
# (1) If you know the name of the model, you can also just use the get_model() convenience function above.
# In this example, the model has been uploaded to wallaroo with the name "mymodel"

model = get_model("mymodel") 

# (2) To get the model without knowing its name (for a single-step pipeline)
model = pipeline.model_configs()[0].model()


# specify the bounds
hi_bnd = 1500000.0 # 1.5M

#
# some examples of validation rules
#

# (1)  validation rule: prediction should be < 1.5 million
pipeline = pipeline.add_validation("less than 1.5m", model.outputs[0][0] < hi_bnd)

# deploy the pipeline to set the steps
pipeline.deploy()

```

When data is passed to the pipeline for inferences, the pipeline will log a check failure whenever one of the validation expressions evaluates to false. Here are some examples of inference results from a pipeline with the validation rule `model.outputs[0][0] < 400000.0`.

![Inferences with check failures](./images/validation_results.png)

You can also find check failures in the logs:

```python
logs = pipeline.logs()
logs.loc[logs['check_failures'] > 0]
```

### Model Validation Rules Exercise

Add some simple validation rules to the model pipeline that you created in a previous exercise.

* Add an upper bound or a lower bound to the model predictions
* Try to create predictions that fall both in and out of the specified range
* Look through the logs to find the check failures.

**HINT 1**: since the purpose of this exercise is try out validation rules, it might be a good idea to take a small data set and make predictions on that data set first, *then* set the validation rules based on those predictions, so that you can see the check failures trigger.

Here's an example:

```python
hi_bnd = 600000.0

pipeline = pipeline.add_validation("less than 600k", prime_model_version.outputs[0][0] < hi_bnd)

pipeline.deploy()

multiple_result = pipeline.infer_from_file('../data/test_data.df.json')

display(multiple_result[multiple_result['check_failures'] > 0])
```

In [21]:
# blank space to set a validation rule on the pipeline and check if it triggers as expected

# 50%
hi_bnd = 50.0

pipeline = pipeline.add_validation("less than 50%", sentiment_model_version.outputs[0][0] > hi_bnd / 100)

pipeline.deploy()

multiple_result = pipeline.infer_from_file('../data/test_data.df.json')

display(multiple_result[multiple_result['check_failures'] > 0])
display(len(multiple_result[multiple_result['check_failures'] > 0]))

,time,in.tensor,out.dense_1,check_failures
0,2023-09-19 15:53:02.990,"[1607.0, 2635.0, 5749.0, 199.0, 49.0, 351.0, 16.0, 2919.0, 159.0, 5092.0, 2457.0, 8.0, 11.0, 1252.0, 507.0, 42.0, 287.0, 316.0, 15.0, 65.0, 136.0, 2.0, 133.0, 16.0, 4311.0, 131.0, 286.0, 153.0, 5.0, 2826.0, 175.0, 54.0, 548.0, 48.0, 1.0, 17.0, 9.0, 183.0, 1.0, 111.0, 15.0, 1.0, 17.0, 284.0, 982.0, 18.0, 28.0, 211.0, 1.0, 1382.0, 8.0, 146.0, 1.0, 19.0, 12.0, 9.0, 13.0, 21.0, 1898.0, 122.0, 14.0, 70.0, 14.0, 9.0, 97.0, 25.0, 74.0, 1.0, 189.0, 12.0, 9.0, 6.0, 31.0, 3.0, 244.0, 2497.0, 3659.0, 2.0, 665.0, 2497.0, 63.0, 180.0, 1.0, 17.0, 6.0, 287.0, 3.0, 646.0, 44.0, 15.0, 161.0, 50.0, 71.0, 438.0, 351.0, 31.0, 5749.0, 2.0, 0.0, 0.0]",[0.37142318],1
2,2023-09-19 15:53:02.990,"[8.0, 3637.0, 4293.0, 1.0, 4523.0, 2.0, 2869.0, 4402.0, 2312.0, 8937.0, 4895.0, 3.0, 1872.0, 2204.0, 4.0, 695.0, 8461.0, 40.0, 5.0, 76.0, 192.0, 275.0, 5.0, 3164.0, 2.0, 8.0, 1813.0, 6501.0, 24.0, 4095.0, 2.0, 61.0, 5182.0, 6.0, 484.0, 2.0, 257.0, 1276.0, 16.0, 1639.0, 369.0, 7.0, 7.0, 6.0, 32.0, 2497.0, 1147.0, 2.0, 573.0, 355.0, 17.0, 41.0, 32.0, 9111.0, 3729.0, 12.0, 492.0, 3.0, 376.0, 4.0, 501.0, 39.0, 2477.0, 40.0, 5.0, 76.0, 192.0, 275.0, 5.0, 815.0, 4119.0, 2.0, 109.0, 1238.0, 3422.0, 685.0, 5.0, 24.0, 8670.0, 1997.0, 8.0, 16.0, 894.0, 11.0, 106.0, 59.0, 27.0, 1.0, 2606.0, 6725.0, 3528.0, 4.0, 1.0, 2132.0, 1391.0, 2.0, 445.0, 20.0, 11.0, 62.0]",[0.07601619],1
3,2023-09-19 15:53:02.990,"[11.0, 19.0, 6.0, 364.0, 16.0, 3452.0, 7794.0, 2.0, 196.0, 105.0, 560.0, 3.0, 173.0, 5.0, 27.0, 4624.0, 8.0, 1.0, 93.0, 4.0, 65.0, 285.0, 83.0, 196.0, 105.0, 23.0, 52.0, 462.0, 2033.0, 228.0, 9.0, 251.0, 5.0, 64.0, 616.0, 802.0, 1445.0, 330.0, 1080.0, 19.0, 45.0, 2571.0, 2.0, 22.0, 23.0, 914.0, 5.0, 103.0, 3.0, 2273.0, 19.0, 148.0, 4451.0, 124.0, 303.0, 5.0, 25.0, 3.0, 125.0, 4453.0, 1274.0, 10.0, 207.0, 2784.0, 2571.0, 18.0, 15.0, 1.0, 695.0, 43.0, 47.0, 11.0, 215.0, 3.0, 436.0, 131.0, 285.0, 709.0, 187.0, 23.0, 21.0, 1.0, 2154.0, 4.0, 1.0, 201.0, 19.0, 1184.0, 40.0, 1.0, 7076.0, 5002.0, 109.0, 686.0, 2745.0, 300.0, 7.0, 7.0, 14.0, 15.0]",[0.24645236],1
4,2023-09-19 15:53:02.990,"[10.0, 61.0, 927.0, 20.0, 63.0, 52.0, 49.0, 105.0, 2.0, 20.0, 2148.0, 1906.0, 58.0, 5517.0, 6.0, 5.0, 336.0, 81.0, 34.0, 178.0, 5.0, 64.0, 84.0, 105.0, 5.0, 1138.0, 65.0, 55.0, 2.0, 275.0, 6528.0, 7.0, 7.0, 10.0, 79.0, 178.0, 5.0, 567.0, 81.0, 3109.0, 65.0, 55.0, 20.0, 1240.0, 2.0, 178.0, 5.0, 1.0, 189.0, 12.0, 1.0, 164.0, 1323.0, 4.0, 131.0, 1240.0, 105.0, 188.0, 76.0, 242.0, 16.0, 9.0, 15.0, 52.0, 193.0, 72.0, 77.0, 166.0, 43.0, 34.0, 22.0, 23.0, 2.0, 77.0, 2299.0, 16.0, 43.0, 2187.0, 2.0, 7.0, 7.0, 11.0, 19.0, 692.0, 732.0, 80.0, 1.0, 1240.0, 2375.0, 7.0, 7.0, 1.0, 164.0, 2.0, 561.0, 6.0, 305.0, 42.0, 207.0, 3.0]",[0.08632833],1
6,2023-09-19 15:53:02.990,"[2108.0, 143.0, 803.0, 18.0, 57.0, 15.0, 1.0, 1351.0, 195.0, 40.0, 93.0, 96.0, 952.0, 5.0, 27.0, 2599.0, 626.0, 22.0, 67.0, 1338.0, 1.0, 201.0, 1.0, 1029.0, 558.0, 85.0, 9.0, 13.0, 151.0, 99.0, 2.0, 245.0, 284.0, 18.0, 195.0, 74.0, 221.0, 277.0, 35.0, 1.0, 751.0, 884.0, 5.0, 27.0, 3.0, 114.0, 326.0, 1247.0, 21.0, 57.0, 50.0, 7546.0, 935.0, 1.0, 83.0, 17.0, 66.0, 1.0, 1368.0, 4.0, 1375.0, 7074.0, 3.0, 129.0, 34.0, 97.0, 57.0, 94.0, 6312.0, 8.0, 5715.0, 303.0, 6977.0, 12.0, 11.0, 28.0, 5928.0, 1501.0, 35.0, 47.0, 13.0, 54.0, 8.0, 230.0, 12.0, 572.0, 9.0, 40.0, 465.0, 707.0, 7.0, 7.0, 3388.0, 10.0, 61.0, 216.0, 11.0, 277.0, 51.0, 10.0]",[0.024733633],1
10,2023-09-19 15:53:02.990,"[246.0, 17.0, 123.0, 107.0, 246.0, 113.0, 96.0, 10.0, 562.0, 836.0, 3.0, 17.0, 430.0, 92.0, 11.0, 161.0, 5.0, 64.0, 54.0, 113.0, 30.0, 29.0, 827.0, 1398.0, 153.0, 141.0, 165.0, 15.0, 157.0, 289.0, 10.0, 388.0, 34.0, 13.0, 375.0, 192.0, 5.0, 162.0, 273.0, 275.0, 80.0, 11.0, 17.0, 7.0, 7.0, 143.0, 803.0, 15.0, 2127.0, 3337.0, 212.0, 27.0, 1208.0, 10.0, 562.0, 836.0, 86.0, 24.0, 212.0, 27.0, 5.0, 4357.0, 653.0, 1.0, 289.0, 7.0, 7.0, 1.0, 1914.0, 8.0, 1.0, 17.0, 379.0, 33.0, 125.0, 1226.0, 5.0, 7.0, 7.0,

41

## Clean Up

At this point, if you are not continuing on to the next notebook, undeploy your pipeline to give the resources back to the environment.

In [22]:
## blank space to undeploy the pipeline

pipeline.undeploy()

name,imdb-reviewer
created,2023-09-19 15:48:25.276825+00:00
last_updated,2023-09-19 15:52:55.225396+00:00
deployed,False
tags,
versions,"150ca80d-b9b0-4011-a515-2fb6f04045f8, 2d91092f-d3c6-4694-8c59-e6537efc342a, a5f1d60f-d76b-4c9f-b047-9d8ec04699e3, 8c40e310-b6bc-4f77-8b73-258e0f23cced, bed8c8d4-a67d-4795-b6ee-fcd7d8535bb3"
steps,embedder


## Congratulations!

In this workshop you have

* Set a validation rule on your house price prediction pipeline.
* Detected model predictions that failed the validation rule.

In the next notebook, you will learn how to monitor the distribution of model outputs for drift away from expected behavior.